In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
device = torch.device('cuda')

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/fer2013.csv')

df['pixels'] = df['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype='float32'))


train_df = df[df['Usage'] == 'Training']
test_df = df[df['Usage'] != 'Training']

X_train = torch.from_numpy(np.vstack(train_df['pixels'].values / 255)).view(-1, 48, 48).unsqueeze(1)
X_test = torch.from_numpy(np.vstack(test_df['pixels'].values / 255)).view(-1, 48, 48).unsqueeze(1)

y_train = torch.from_numpy(train_df['emotion'].values)
y_test = torch.from_numpy(test_df['emotion'].values)

y_train = F.one_hot(y_train, num_classes=7).float()
y_test = F.one_hot(y_test, num_classes=7).float()

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
for img, label in train_loader:
    print(img.shape, label.shape)
    break

torch.Size([32, 1, 48, 48]) torch.Size([32, 7])


In [ ]:
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Flatten(),

    nn.Linear(256 * 6 * 6, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(512, 7),
    nn.Softmax(dim=1),
)

model.to(device)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=9216, out_features=1024, bias=True)
  (14): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): ReLU()
  (16

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 30
best_val_acc = 0

for epoch in range(n_epochs):
    model.train()
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        pred = model(images)
        loss = loss_fn(pred, labels)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == labels.argmax(1)).sum()
        total += labels.size(0)
    train_acc = correct / total

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)
            loss = loss_fn(pred, labels)

            correct += (pred.argmax(1) == labels.argmax(1)).sum()
            total += labels.size(0)
    test_acc = correct / total

    print(f'epoch [{epoch + 1}/{n_epochs}], train acc: {train_acc * 100:.1f}%, test acc: {test_acc * 100:.1f}%')

    if test_acc >= best_val_acc:
        best_val_acc = test_acc
        torch.save(model, 'emotion_model.pth')
        print('best model saved')

100%|██████████| 225/225 [00:00<00:00, 405.84it/s]


epoch [1/30], train acc: 39.0%, test acc: 44.9%
best model saved


100%|██████████| 225/225 [00:00<00:00, 414.80it/s]


epoch [2/30], train acc: 47.5%, test acc: 49.1%
best model saved


100%|██████████| 225/225 [00:00<00:00, 406.30it/s]


...

epoch [28/30], train acc: 80.7%, test acc: 58.4%


100%|██████████| 225/225 [00:00<00:00, 395.81it/s]


epoch [29/30], train acc: 81.4%, test acc: 59.1%
best model saved


100%|██████████| 225/225 [00:00<00:00, 371.91it/s]

epoch [30/30], train acc: 82.3%, test acc: 58.2%
